# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe598885f70>
├── 'a' --> tensor([[ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771]])
└── 'x' --> <FastTreeValue 0x7fe598885e50>
    └── 'c' --> tensor([[-0.2583, -0.4552, -1.0148,  0.1077],
                        [-1.0593, -1.1693, -0.2817, -1.1155],
                        [-1.3689,  0.9352,  0.0687,  1.7535]])

In [4]:
t.a

tensor([[ 1.8840,  0.6358,  0.8807],
        [-0.6081,  0.6676, -0.6771]])

In [5]:
%timeit t.a

65.1 ns ± 0.726 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe598885f70>
├── 'a' --> tensor([[-1.4477,  0.7728,  1.6120],
│                   [-0.6420, -0.1821, -0.4794]])
└── 'x' --> <FastTreeValue 0x7fe598885e50>
    └── 'c' --> tensor([[-0.2583, -0.4552, -1.0148,  0.1077],
                        [-1.0593, -1.1693, -0.2817, -1.1155],
                        [-1.3689,  0.9352,  0.0687,  1.7535]])

In [7]:
%timeit t.a = new_value

65.7 ns ± 0.0813 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771]]),
    x: Batch(
           c: tensor([[-0.2583, -0.4552, -1.0148,  0.1077],
                      [-1.0593, -1.1693, -0.2817, -1.1155],
                      [-1.3689,  0.9352,  0.0687,  1.7535]]),
       ),
)

In [10]:
b.a

tensor([[ 1.8840,  0.6358,  0.8807],
        [-0.6081,  0.6676, -0.6771]])

In [11]:
%timeit b.a

56.1 ns ± 0.029 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0069,  0.3632, -1.2703],
               [ 0.5003,  0.0444,  0.9491]]),
    x: Batch(
           c: tensor([[-0.2583, -0.4552, -1.0148,  0.1077],
                      [-1.0593, -1.1693, -0.2817, -1.1155],
                      [-1.3689,  0.9352,  0.0687,  1.7535]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.0993 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

958 ns ± 1.92 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 291 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 267 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe4f316a760>
├── 'a' --> tensor([[[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]],
│           
│                   [[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]],
│           
│                   [[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]],
│           
│                   [[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]],
│           
│                   [[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]],
│           
│                   [[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]],
│           
│                   [[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]],
│           
│                   [[ 1.8840,  0.6358,  0.8807],
│                    [-0.6081,  0.6676, -0.6771]]])
└── 'x' --> <FastTreeValue 0x7fe59b949940>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe598885a30>
├── 'a' --> tensor([[ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771],
│                   [ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771],
│                   [ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771],
│                   [ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771],
│                   [ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771],
│                   [ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771],
│                   [ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771],
│                   [ 1.8840,  0.6358,  0.8807],
│                   [-0.6081,  0.6676, -0.6771]])
└── 'x' --> <FastTreeValue 0x7fe598885e80>
    └── 'c' --> tensor([[-0.2583, -0.4552, -1.0148,  0.1077],
                        [-1.0593, -1.1693, -0.2817, -1.1155],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 37.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 152 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]],
       
               [[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]],
       
               [[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]],
       
               [[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]],
       
               [[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]],
       
               [[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]],
       
               [[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]],
       
               [[ 1.8840,  0.6358,  0.8807],
                [-0.6081,  0.6676, -0.6771]]]),
    x: Batch(
           c: tensor([[[-0.2583, -0.4552, -1.0148,  0.1077],
                       [-1.0593, -1.1693, -0.2817, -1.1155],
                       [-1.3689,  0.9352,  0.0687,  1.7535]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 69.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771],
               [ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771],
               [ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771],
               [ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771],
               [ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771],
               [ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771],
               [ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771],
               [ 1.8840,  0.6358,  0.8807],
               [-0.6081,  0.6676, -0.6771]]),
    x: Batch(
           c: tensor([[-0.2583, -0.4552, -1.0148,  0.1077],
                      [-1.0593, -1.1693, -0.2817, -1.1155],
                      [-1.3689,  0.9352,  0.0687,  1.7535],
                      [-0.2583, -0.4552, -1.0148,  0.1077],
                      [-1.0593, -

In [28]:
%timeit Batch.cat(batches)

141 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 4.59 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
